In [2]:
import pandas as pd
import geopandas as gpd
import itertools
import seaborn as sns

import matplotlib.pyplot as plt
from dbfread import DBF
import numpy as np

c:\Users\cday\Anaconda3\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\cday\AppData\Local\Temp\ipykernel_9424\2937851097.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are u

In [3]:
# import global TDM functions
import sys
sys.path.insert(0, '../Resources/2-Python/global-functions')
import BigQuery

client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()

In [38]:
hts_hh_23 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.hts_2023_tdm_proc.hh').to_dataframe()
hts_trip_23 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.hts_2023_tdm_proc.trip').to_dataframe()

hts_hh_12 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.previous_hts_2012_v30.household').to_dataframe()
hts_trip_12 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.previous_hts_2012_v30.trip').to_dataframe()

bq_taz_ustm3 = client.query("SELECT * FROM " + 'confidential-2023-utah-hts.geometries.ustm_v3_taz_2021_09_22_geog').to_dataframe()
gdf_taz = gpd.GeoDataFrame(bq_taz_ustm3)

#hts_hh_12 = pd.read_csv(r'source_data_archive/hts-2012/HTS 2012 - db - Household.csv')
#hts_trip_12 = pd.read_csv(r'source_data_archive/hts-2012/HTS 2012 - db - Trip.csv')

In [39]:
gdf_v910_taz = gpd.read_file('data/TAZ/WFv910_TAZ.shp')[['TAZID','CO_TAZID']]

## Observed TLF Calculation (Previous Method)
The previous method to calculating observed TLF is documented on Model Ace: A:\1 - TDM\2 - Estimate Param\_General Parameters\7 - Update Obs TLF
 0. Data prep (calculate new columns to allow easy processing)
 1. Create Weighted Average Daily Skims
     - using period skims from a recent model run as well as the diurnal pa factors csv file, calculate an average daily distance, general cost, and time skim (voyager script used for this step)
 2. Prepare Truck TLF Data (not explained here)
 3. Prepared Other Purpose TLF Data
     - merge the daily skim data (distance, generalized cost, and time) with the household travel survey trip data
     - create binned table of data
 4. Create Obs TLFs for all trips
     - merge the truck TLF data with the other purpose TLF data
 5. Smooth in Excel
      - the data is then read into excel in undergoes various rounds of smoothing/normalizing/hand adjusting to ensure each TLF curve is smooth, maintains the same average, and looks pretty

## Observed TLF Calculation (New/Proposed Method)
The new/proposed method involves generating PA matrix tables directly from the household travel survey and then letting the model calculate the TLF curves with its current voyager code (which calculates modeled TLF curves). In other words observed "modeled" TLF curves. To do this we will do the following steps:
 0. Data prep (calculate new columns to allow easy processing)
 1. Prepare non-truck non-external PA matrices from the HTS
 2. Prepare external PA matrices from the HTS
      - this involves matching external stations to each IXXI trip
 3. Adjust 4_PA_Distrib_TLF.s to read in new PA observed matrices
 4. run new 4_PA_Distrib_TLF.s to generate observed TLF curves

 5. Prepare truck stuff later



### Data Prep

In [9]:
# merge gdf to get taz level insights
hts_hh_23_merge = hts_hh_23.copy()[['hh_id','hCO_TAZID_USTMv3']]
hts_trip_1 = hts_trip_23.copy().merge(hts_hh_23_merge,how='left',left_on='hh_id',right_on='hh_id')

# household
hts_trip_1 = hts_trip_1.copy().merge(gdf_v910_taz,how='left',left_on='hCO_TAZID_USTMv3',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'hTAZID'})
hts_trip_1['hTAZID'].fillna(-1, inplace=True)

# origin
hts_trip_1 = hts_trip_1.copy().merge(gdf_v910_taz,how='left',left_on='oCO_TAZID_USTMv3',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'oTAZID'})
hts_trip_1['oTAZID'].fillna(-1, inplace=True)

# destination
hts_trip_1 = hts_trip_1.merge(gdf_v910_taz,how='left',left_on='dCO_TAZID_USTMv3',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'dTAZID'})
hts_trip_1['dTAZID'].fillna(-1, inplace=True)

# production
hts_trip_1 = hts_trip_1.merge(gdf_v910_taz,how='left',left_on='pCO_TAZID_USTMv3',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'pTAZID'})
hts_trip_1['pTAZID'].fillna(-1, inplace=True)

# attraction
hts_trip_1 = hts_trip_1.merge(gdf_v910_taz,how='left',left_on='aCO_TAZID_USTMv3',right_on='CO_TAZID')
hts_trip_1 = hts_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'aTAZID'})
hts_trip_1['aTAZID'].fillna(-1, inplace=True)

hts_trip_1 = hts_trip_1[['PURP7_t','HBSch_lev','PA_AP','hTAZID','oTAZID','dTAZID','pTAZID','aTAZID','trip_weight_v2']]
hts_trip_1 = hts_trip_1[hts_trip_1['PURP7_t']!='Missing Response']
hts_trip_1

C:\Users\cday\AppData\Local\Temp\ipykernel_9424\2006052473.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hts_trip_1['hTAZID'].fillna(-1, inplace=True)
C:\Users\cday\AppData\Local\Temp\ipykernel_9424\2006052473.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,trip_weight_v2
1,NHBW,None,PA,-1.0,-1.0,-1.0,-1.0,-1.0,229.774490
2,HBShp,None,AP,-1.0,-1.0,-1.0,-1.0,-1.0,427.210348
3,HBOth,None,PA,521.0,521.0,441.0,521.0,441.0,0.000000
4,NHBNW,None,PA,521.0,441.0,437.0,441.0,437.0,0.000000
5,NHBNW,None,PA,521.0,437.0,437.0,437.0,437.0,0.000000
...,...,...,...,...,...,...,...,...,...
389084,NHBNW,None,PA,646.0,703.0,614.0,703.0,614.0,201.609649
389085,NHBNW,None,PA,646.0,614.0,645.0,614.0,645.0,201.609649
389086,HBShp,None,AP,646.0,645.0,646.0,646.0,645.0,440.439540
389087,HBW,None,PA,646.0,646.0,646.0,646.0,646.0,179.438331


In [10]:
# calculate WF_IIIXXIXX
# Categories:
# - II_Res: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - II_Vis: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - IX_AP:  hTAZID != -1  --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
# - IX_PA:  hTAZID != -1  --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_AP:  hTAZID = -1   --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_PA:  hTAZID = -1   --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
hts_trip_2 = hts_trip_1.copy()

h_res = hts_trip_2["hTAZID"] != -1
v_res = hts_trip_2["hTAZID"] == -1
p_in  = hts_trip_2["pTAZID"] != -1
a_in  = hts_trip_2["aTAZID"] != -1
o_in  = hts_trip_2["oTAZID"] != -1
d_in  = hts_trip_2["dTAZID"] != -1

# Conditions (order matters: first match wins)
cond_ii_res = h_res & p_in & a_in & o_in & d_in
cond_ii_vis = v_res & p_in & a_in & o_in & d_in
condi_xx = (~h_res) & (~p_in) &  (~a_in) & (~o_in) & (~d_in)

# IX (resident)
cond_ix_ap = h_res & (
    ((hts_trip_2["PA_AP"] == "PA") & (~p_in) & a_in) |   # PA: produced outside, attracted inside
    ((hts_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)
cond_ix_pa = h_res & (
    ((hts_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)

# XI (non-resident)
cond_xi_ap = (~h_res) & (
    ((hts_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)
cond_xi_pa = (~h_res) & (
    ((hts_trip_2["PA_AP"] == "PA") & (~p_in) &  a_in) |  # PA: produced outside, attracted inside
    ((hts_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)


conds = [cond_ii_res, cond_ii_vis, cond_ix_ap, cond_ix_pa, cond_xi_ap, cond_xi_pa, condi_xx]
labels = ["II_Res", "II_Vis", "IX_AP", "IX_PA", "XI_AP", "XI_PA", 'XX']

hts_trip_2["WF_IIIXXIXX"] = np.select(conds, labels, default=np.nan)

In [11]:
hts_trip_3 = hts_trip_2.copy()
hts_trip_3['PURP'] = np.where(
    hts_trip_3['PURP7_t'] == 'HBSch',
    hts_trip_3['HBSch_lev'].map({'primary': 'HBSch_Pr', 'secondary': 'HBSch_Sc'}),
    hts_trip_3['PURP7_t']
)
hts_trip_3

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,trip_weight_v2,WF_IIIXXIXX,PURP
1,NHBW,None,PA,-1.0,-1.0,-1.0,-1.0,-1.0,229.774490,XX,NHBW
2,HBShp,None,AP,-1.0,-1.0,-1.0,-1.0,-1.0,427.210348,XX,HBShp
3,HBOth,None,PA,521.0,521.0,441.0,521.0,441.0,0.000000,II_Res,HBOth
4,NHBNW,None,PA,521.0,441.0,437.0,441.0,437.0,0.000000,II_Res,NHBNW
5,NHBNW,None,PA,521.0,437.0,437.0,437.0,437.0,0.000000,II_Res,NHBNW
...,...,...,...,...,...,...,...,...,...,...,...
389084,NHBNW,None,PA,646.0,703.0,614.0,703.0,614.0,201.609649,II_Res,NHBNW
389085,NHBNW,None,PA,646.0,614.0,645.0,614.0,645.0,201.609649,II_Res,NHBNW
389086,HBShp,None,AP,646.0,645.0,646.0,646.0,645.0,440.439540,II_Res,HBShp
389087,HBW,None,PA,646.0,646.0,646.0,646.0,646.0,179.438331,II_Res,HBW


### Prepare non-truck non-external PA matrices from the HTS

In [12]:
df_tlf_1 = hts_trip_3.copy()

# filter out XX and nan 
df_tlf_1 = df_tlf_1[df_tlf_1['WF_IIIXXIXX']!='XX']
df_tlf_1 = df_tlf_1[df_tlf_1['WF_IIIXXIXX']!='nan']

# adjust to only PA direction
df_tlf_1['p_taz'] = np.where(df_tlf_1['PA_AP']=='PA', df_tlf_1['pTAZID'], df_tlf_1['aTAZID']).astype(int)
df_tlf_1['a_taz'] = np.where(df_tlf_1['PA_AP']=='PA', df_tlf_1['aTAZID'], df_tlf_1['pTAZID']).astype(int)
df_tlf_1 = df_tlf_1[['WF_IIIXXIXX','PURP','p_taz','a_taz','trip_weight_v2']]
df_tlf_1 = df_tlf_1[df_tlf_1['trip_weight_v2']>0]
df_tlf_1['trip_weight_v2'] = df_tlf_1['trip_weight_v2'].round(2)
df_tlf_1

,WF_IIIXXIXX,PURP,p_taz,a_taz,trip_weight_v2
34,II_Res,HBOth,521,604,400.77
35,II_Res,HBOth,604,521,400.77
36,II_Res,HBW,521,559,400.77
37,II_Res,HBW,559,521,400.77
54,II_Res,HBOth,325,273,2.44
...,...,...,...,...,...
389084,II_Res,NHBNW,703,614,201.61
389085,II_Res,NHBNW,614,645,201.61
389086,II_Res,HBShp,645,646,440.44
389087,II_Res,HBW,646,646,179.44


In [15]:
import pandas as pd
import numpy as np
import openmatrix as omx

# Filter trips for the II_Res and II_Vis groups
df_tlf_ii = df_tlf_1[df_tlf_1['WF_IIIXXIXX'].isin(['II_Res','II_Vis'])].copy()
df_tlf_ii["trip_weight_v2"] = df_tlf_ii["trip_weight_v2"].fillna(0)

# Only take unique TAZs
unique_taz = np.sort(np.union1d(df_tlf_ii["p_taz"].unique(), df_tlf_ii["a_taz"].unique()))
n_taz = len(unique_taz)
taz_map = {orig: new for new, orig in enumerate(unique_taz)}  # map to 0..n_taz-1

# Open Cube-compatible OMX (legacy v1.2)
with omx.open_file("obs_pa_table_2023.omx", "w", version="1.2") as omx_file:
    omx_file.create_mapping("TAZ", list(range(1, n_taz+1)))

    # Initialize total matrix
    total_mat = np.zeros((n_taz, n_taz), dtype=np.float64)

    # First pass: accumulate total matrix only
    for _, g in df_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["trip_weight_v2"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["trip_weight_v2"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        total_mat += mat

    # Write TOT tab first
    omx_file["TOT"] = total_mat

    # Second pass: write individual purpose tabs
    for purp, g in df_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["trip_weight_v2"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["trip_weight_v2"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        omx_file[str(purp).upper()] = mat

print("OMX file created successfully with TOT tab first.")


OMX file created successfully with TOT tab first.


In [14]:
df_tlf_ii[df_tlf_ii['PURP']=='HBSch_Sc']['trip_weight_v2'].sum()

130192.1

## Recalcualte 2012 Observed with new Methodology

### Data Prep

In [51]:
# household
hts12_trip_1 = hts_trip_12.copy().merge(gdf_v910_taz,how='left',left_on='h_CoTAZID_v30',right_on='CO_TAZID')
hts12_trip_1 = hts12_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'hTAZID'})
hts12_trip_1['hTAZID'].fillna(-1, inplace=True)

# origin
hts12_trip_1 = hts12_trip_1.copy().merge(gdf_v910_taz,how='left',left_on='o_CoTAZID_v30',right_on='CO_TAZID')
hts12_trip_1 = hts12_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'oTAZID'})
hts12_trip_1['oTAZID'].fillna(-1, inplace=True)

# destination
hts12_trip_1 = hts12_trip_1.merge(gdf_v910_taz,how='left',left_on='d_CoTAZID_v30',right_on='CO_TAZID')
hts12_trip_1 = hts12_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'dTAZID'})
hts12_trip_1['dTAZID'].fillna(-1, inplace=True)

# production
hts12_trip_1 = hts12_trip_1.merge(gdf_v910_taz,how='left',left_on='p_CoTAZID_v30',right_on='CO_TAZID')
hts12_trip_1 = hts12_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'pTAZID'})
hts12_trip_1['pTAZID'].fillna(-1, inplace=True)

# attraction
hts12_trip_1 = hts12_trip_1.merge(gdf_v910_taz,how='left',left_on='a_CoTAZID_v30',right_on='CO_TAZID')
hts12_trip_1 = hts12_trip_1.drop(columns=['CO_TAZID']).rename(columns={'TAZID':'aTAZID'})
hts12_trip_1['aTAZID'].fillna(-1, inplace=True)

hts12_trip_1 = hts12_trip_1[['PURP7_t','HBSch_lev','PA_AP','hTAZID','oTAZID','dTAZID','pTAZID','aTAZID','weight']]
hts12_trip_1 = hts12_trip_1[hts12_trip_1['PURP7_t']!='Missing Response']
hts12_trip_1

C:\Users\cday\AppData\Local\Temp\ipykernel_9424\2204456499.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hts12_trip_1['hTAZID'].fillna(-1, inplace=True)
C:\Users\cday\AppData\Local\Temp\ipykernel_9424\2204456499.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,weight
0,NHBNW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,72.796650
1,NHBNW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,69.603200
2,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,43.967912
3,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,55.207550
4,HBOth,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,69.603200
...,...,...,...,...,...,...,...,...,...
99303,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,43.688708
99304,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,61.700187
99305,NHBW,NULL,PA,1758.0,-1.0,-1.0,-1.0,-1.0,116.497519
99306,NHBNW,NULL,PA,1910.0,-1.0,-1.0,-1.0,-1.0,107.696958


In [52]:
# calculate WF_IIIXXIXX
# Categories:
# - II_Res: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - II_Vis: hTAZID != -1, pSUBAREA = 1, aSUBAREA = 1, oTAZID != -1, dTAZID != -1
# - IX_AP:  hTAZID != -1  --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
# - IX_PA:  hTAZID != -1  --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_AP:  hTAZID = -1   --> pa trip produced inside, attracted outside or ap trip produced outside, attracted inside
# - XI_PA:  hTAZID = -1   --> pa trip produced outside, attracted inside or ap trip produced inside, attracted outside
hts12_trip_2 = hts12_trip_1.copy()

h_res = hts12_trip_2["hTAZID"] != -1
v_res = hts12_trip_2["hTAZID"] == -1
p_in  = hts12_trip_2["pTAZID"] != -1
a_in  = hts12_trip_2["aTAZID"] != -1
o_in  = hts12_trip_2["oTAZID"] != -1
d_in  = hts12_trip_2["dTAZID"] != -1

# Conditions (order matters: first match wins)
cond_ii_res = h_res & p_in & a_in & o_in & d_in
cond_ii_vis = v_res & p_in & a_in & o_in & d_in
condi_xx = (~h_res) & (~p_in) &  (~a_in) & (~o_in) & (~d_in)

# IX (resident)
cond_ix_ap = h_res & (
    ((hts12_trip_2["PA_AP"] == "PA") & (~p_in) & a_in) |   # PA: produced outside, attracted inside
    ((hts12_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)
cond_ix_pa = h_res & (
    ((hts12_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts12_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)

# XI (non-resident)
cond_xi_ap = (~h_res) & (
    ((hts12_trip_2["PA_AP"] == "PA") & ( p_in) & ~a_in) |  # PA: produced inside, attracted outside
    ((hts12_trip_2["PA_AP"] == "AP") & (~p_in) &  a_in)    # AP: produced outside, attracted inside
)
cond_xi_pa = (~h_res) & (
    ((hts12_trip_2["PA_AP"] == "PA") & (~p_in) &  a_in) |  # PA: produced outside, attracted inside
    ((hts12_trip_2["PA_AP"] == "AP") & ( p_in) & ~a_in)    # AP: produced inside, attracted outside
)


conds = [cond_ii_res, cond_ii_vis, cond_ix_ap, cond_ix_pa, cond_xi_ap, cond_xi_pa, condi_xx]
labels = ["II_Res", "II_Vis", "IX_AP", "IX_PA", "XI_AP", "XI_PA", 'XX']

hts12_trip_2["WF_IIIXXIXX"] = np.select(conds, labels, default=np.nan)

In [53]:
hts12_trip_3 = hts12_trip_2.copy()
hts12_trip_3['PURP'] = np.where(
    hts12_trip_3['PURP7_t'] == 'HBSch',
    hts12_trip_3['HBSch_lev'].map({'primary': 'HBSch_Pr', 'secondary': 'HBSch_Sc'}),
    hts12_trip_3['PURP7_t']
)
hts12_trip_3

,PURP7_t,HBSch_lev,PA_AP,hTAZID,oTAZID,dTAZID,pTAZID,aTAZID,weight,WF_IIIXXIXX,PURP
0,NHBNW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,72.796650,XX,NHBNW
1,NHBNW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,69.603200,XX,NHBNW
2,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,43.967912,XX,NHBW
3,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,55.207550,XX,NHBW
4,HBOth,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,69.603200,XX,HBOth
...,...,...,...,...,...,...,...,...,...,...,...
99303,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,43.688708,XX,NHBW
99304,NHBW,NULL,PA,-1.0,-1.0,-1.0,-1.0,-1.0,61.700187,XX,NHBW
99305,NHBW,NULL,PA,1758.0,-1.0,-1.0,-1.0,-1.0,116.497519,nan,NHBW
99306,NHBNW,NULL,PA,1910.0,-1.0,-1.0,-1.0,-1.0,107.696958,nan,NHBNW


### Prepare non-truck non-external PA matrices from the HTS

In [54]:
df_tlf12_1 = hts12_trip_3.copy()

# filter out XX and nan 
df_tlf12_1 = df_tlf12_1[df_tlf12_1['WF_IIIXXIXX']!='XX']
df_tlf12_1 = df_tlf12_1[df_tlf12_1['WF_IIIXXIXX']!='nan']

# adjust to only PA direction
df_tlf12_1['p_taz'] = np.where(df_tlf12_1['PA_AP']=='PA', df_tlf12_1['pTAZID'], df_tlf12_1['aTAZID']).astype(int)
df_tlf12_1['a_taz'] = np.where(df_tlf12_1['PA_AP']=='PA', df_tlf12_1['aTAZID'], df_tlf12_1['pTAZID']).astype(int)
df_tlf12_1 = df_tlf12_1[['WF_IIIXXIXX','PURP','p_taz','a_taz','weight']]
df_tlf12_1 = df_tlf12_1[df_tlf12_1['weight']>0]
df_tlf12_1['weight'] = df_tlf12_1['weight'].round(2)
df_tlf12_1

,WF_IIIXXIXX,PURP,p_taz,a_taz,weight
11,IX_PA,NHBW,1106,-1,91.28
12,IX_PA,HBOth,2904,-1,71.44
13,IX_PA,HBOth,2904,-1,71.44
14,IX_PA,HBOth,3374,-1,172.89
15,IX_PA,HBOth,3374,-1,172.89
...,...,...,...,...,...
98969,IX_PA,HBOth,362,-1,227.09
98970,IX_PA,HBOth,563,-1,74.81
98971,IX_PA,HBOth,568,-1,191.37
98972,IX_PA,HBOth,568,-1,191.37


In [55]:
import pandas as pd
import numpy as np
import openmatrix as omx

# Filter trips for the II_Res and II_Vis groups
df12_tlf_ii = df_tlf12_1[df_tlf12_1['WF_IIIXXIXX'].isin(['II_Res','II_Vis'])].copy()
df12_tlf_ii["weight"] = df12_tlf_ii["weight"].fillna(0)

# Only take unique TAZs
unique_taz = np.sort(np.union1d(df12_tlf_ii["p_taz"].unique(), df12_tlf_ii["a_taz"].unique()))
n_taz = len(unique_taz)
taz_map = {orig: new for new, orig in enumerate(unique_taz)}  # map to 0..n_taz-1

# Open Cube-compatible OMX (legacy v1.2)
with omx.open_file("obs_pa_table_2012.omx", "w", version="1.2") as omx_file:
    omx_file.create_mapping("TAZ", list(range(1, n_taz+1)))

    # Initialize total matrix
    total_mat = np.zeros((n_taz, n_taz), dtype=np.float64)

    # First pass: accumulate total matrix only
    for _, g in df12_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["weight"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["weight"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        total_mat += mat

    # Write TOT tab first
    omx_file["TOT"] = total_mat

    # Second pass: write individual purpose tabs
    for purp, g in df12_tlf_ii.groupby("PURP"):
        g = g[g["p_taz"].isin(unique_taz) & g["a_taz"].isin(unique_taz)]
        g_agg = g.groupby(["p_taz","a_taz"])["weight"].sum().reset_index()
        rows = g_agg["p_taz"].map(taz_map).values
        cols = g_agg["a_taz"].map(taz_map).values
        vals = g_agg["weight"].values.astype(np.float64)
        mat = np.zeros((n_taz, n_taz), dtype=np.float64)
        mat[rows, cols] = vals
        omx_file[str(purp).upper()] = mat

print("OMX file created successfully with TOT tab first.")


OMX file created successfully with TOT tab first.


In [64]:
df12_tlf_ii[df12_tlf_ii['PURP']=='HBSch_Sc']['weight'].sum()

99435.45999999999